In [1]:
import pandas as pd
import numpy as np
import json

from data_processing.readers.economic_data.eurostat_reader import EuroStatReader
from data_processing.readers.economic_data.exchange_rates import WorldBankRatesReader
from data_processing.readers.economic_data.oecd_economic_data import OECDEconData
from data_processing.readers.economic_data.policy_rates import PolicyRatesReader
from data_processing.readers.economic_data.world_bank_reader import WorldBankReader

from data_processing.readers.io_tables.icio_reader import ICIOReader
from data_processing.readers.io_tables.wiod_reader import WIODReader

from data_processing.readers.population_data.employment_data import WorldBankEmployment
from data_processing.readers.population_data.hfcs_reader import HFCSReader
from data_processing.readers.population_data.pop_sampler import OECDPopulationSampler

from data_processing.readers.socioeconomic_data.wiod_sea_data import WIODSEAReader

In [2]:
years = np.arange(2000,2022)
countries = [
     "GBR", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", 
    "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", 
    "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE", 
    "ZAF", "USA", "GBR", "BRA"
]

eurostat = [
    "firm_debt_ratios",
    "firm_deposit_ratios",
    "cb_debt_ratios",
    "cb_equity_ratios",
    "general_gov_debt_ratios",
    "central_gov_debt_ratios",
    "shortterm_gov_bond_rates",
    "longterm_gov_bond_rates",
    "dividend_payout_ratio",
    "quarterly_gdp",
    "quarterly_inflation",
]

wb_exchange_rates = [
    "to_usd"
]

oecd = [
    "employees_by_industry",
    # "read_business_demography",
    "read_tau_sif",
    "read_tau_siw",
    "read_tau_firm",
    "read_tau_income",
    "read_immediate_interest_rates",
    "read_shortterm_interest_rates",
    "read_longterm_interest_rates",
    "read_tierone_reserves",
    "unemployment_benefits_gdp_pct",
    "all_cash_benefits_gdp_pct"
]

bis = [
    "cb_policy_rate"
]

wb = [
    "unemployment_rates",
    "participation_rates",
    "get_tau_vat",
    "get_tau_exp",
    "get_historic_gdp",
    "get_historic_cpi",
    "get_gini_coef",
    "get_current_quarterly_gdp"
]

In [3]:
eurostat_reader = EuroStatReader('raw_data/eurostat.h5', 'raw_data/codes.csv')
wbr_reader = WorldBankRatesReader(pd.read_csv('raw_data/exchange_rates.csv', index_col=0))
oecd_reader = OECDEconData('raw_data/oecd_econ_data.h5', 100)
bis_reader = PolicyRatesReader('raw_data/bis_cb_policy_rates.csv', 'raw_data/codes.csv')
wb_reader = WorldBankReader('raw_data/world_bank.h5')

def handle_tables(param, countries, years, reader):

    matrix = np.zeros((len(countries), len(years)))
    for c_i, c in enumerate(countries): 
        for y_i, y in enumerate(years):

            if param == "shortterm_gov_bond_rates":
                try:
                    method = getattr(type(reader), param)
                    val = method(reader, country=c, year=y, months=0)
                except:
                    val = None

            elif param in ["quarterly_gdp", "quarterly_inflation"]: 

                try:
                    method = getattr(type(reader), param)
                    val = method(reader, country=c, year=y, quarter=1)
                except:
                    val = None

            elif param in ["unemployment_rates","participation_rates"]:

                try:
                    method=getattr(type(reader), param)
                    val = method(reader, year=y)[c]
                except:
                    val = None

            elif param in  ["get_historic_gdp","get_historic_cpi"]:

                try:
                    method=getattr(type(reader), param)
                    val = method(reader, country=c)[y]
                except:
                    val = None

            else:

                try:
                    method = getattr(type(reader), param)
                    val = method(reader, country=c, year=y)
                    if not isinstance(val, (int, float, np.floating, np.integer)): val = 1
                except:
                    val = None
            
            try:
                matrix[c_i, y_i] = val
            except:
                print(val)
                print(param)
                raise Error

    return matrix.tolist()

def get_eurostat(parameters, countries, years):

    params = {}
    for p in parameters:
        params[p + "_eurostat"] = handle_tables(p, countries, years, eurostat_reader)

    return params

def get_wb_exchange_rates(parameters, countries, years):

    params = {}
    for p in parameters:
        params[p + "_worldbank"] = handle_tables(p, countries, years, wbr_reader)

    return params

def get_oecd(parameters, countries, years):

    params = {}
    for p in parameters:
        params[p + "_oecd"] = handle_tables(p, countries, years, oecd_reader)

    return params


def get_bis(parameters, countries, years):

    params = {}
    for p in parameters:
        params[p + "_bis"] = handle_tables(p, countries, years, bis_reader)

    return params

def get_wb(parameters, countries, years):

    params = {}
    for p in parameters:
        params[p + "_worldbank"] = handle_tables(p, countries, years, wb_reader)

    return params

In [4]:
wb_reader.unemployment_rates(2020)["GBR"]

0.0447200012207031

In [5]:
eurostat_results = get_eurostat(eurostat, countries, years)
print("1")
wb_er_results = get_wb_exchange_rates(wb_exchange_rates, countries, years)
print("2")
oecd_results = get_oecd(oecd, countries, years)
print("3")
bis_results = get_bis(bis, countries, years)
print("4")
wb_results = get_wb(wb, countries, years)
print("5")
results = {**eurostat_results, **wb_er_results, **oecd_results, **bis_results, **wb_results}

1
2


/Users/matthewbone/Documents/Coding/VSCode/macro-data-quality/.venv/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/matthewbone/Documents/Coding/VSCode/macro-data-quality/.venv/lib/python3.11/site-packages/data_processing/readers/economic_data/oecd_economic_data.py:84: RuntimeWarning: invalid value encountered in cast
  return col_data.values.astype("int") / self.scale
/Users/matthewbone/Documents/Coding/VSCode/macro-data-quality/.venv/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/matthewbone/Documents/Coding/VSCode/macro-data-quality/.venv/lib/python3.11/site-packages/data_processing/readers/economic_data/oecd_economic_data.py:84: RuntimeWarning: invalid value encountered in cast
  return col_data.values.astype("int") / self.scale
/Users/matthewbone/Documents/C

3
4
5


In [6]:
results

{'firm_debt_ratios_eurostat': [[0.509,
   0.5529999999999999,
   0.578,
   0.556,
   0.56,
   0.615,
   0.65,
   0.662,
   0.762,
   0.703,
   0.66,
   0.62,
   0.639,
   0.598,
   0.5529999999999999,
   0.54,
   0.56,
   0.5760000000000001,
   0.5720000000000001,
   0.532,
   nan,
   nan],
  [1.015,
   1.092,
   1.062,
   1.121,
   1.1340000000000001,
   1.101,
   1.0979999999999999,
   1.187,
   1.3780000000000001,
   1.421,
   1.301,
   1.3730000000000002,
   1.393,
   1.386,
   1.3659999999999999,
   1.374,
   1.555,
   1.432,
   1.375,
   1.379,
   1.435,
   1.348],
  [0.243,
   0.272,
   0.316,
   0.4,
   0.523,
   0.648,
   0.823,
   1.071,
   1.189,
   1.187,
   1.147,
   1.079,
   1.08,
   1.0859999999999999,
   1.097,
   0.924,
   0.863,
   0.8140000000000001,
   0.805,
   0.757,
   0.745,
   0.669],
  [0.43700000000000006,
   0.5379999999999999,
   0.562,
   0.5539999999999999,
   0.5720000000000001,
   0.591,
   0.643,
   0.7040000000000001,
   0.779,
   0.889,
   0.971,
  

In [7]:
results['countries'] = countries
results['years'] = years.tolist()

with open("test_data.json", "w") as f:
    f.write(json.dumps(results))